### Count Vectorization
The easiest way to represent a document as a vector is with a **bag of words Count Vectorizer**. This will turn each document to be a 1-D array, which I think is a good starting point for my clustering algorithms to work. Let's see how it works. 

The Count Vectorizer exclusively accepts series in string format, not as tokenized lists. Each row, representing a document, must be a string. This ensures that every point in the clustering encapsulates an entire sequence, rather than isolated vectorized words.

I set the `min_df` parameter to be equal to 5 to only include terms that occur more than 5 times in my **count vectorized** data. 

In [55]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt 
from sklearn.feature_extraction.text import TfidfVectorizer

ModuleNotFoundError: No module named 'matplotlib'

In [20]:
# %load C:\Sagar Study\ML and Learning\Projects\customer-support-bot\amazon_customer_support\test_environment.py
import sys
import numpy as np 

REQUIRED_PYTHON = "python3"


def main():
    system_major = sys.version_info.major
    if REQUIRED_PYTHON == "python":
        required_major = 2
    elif REQUIRED_PYTHON == "python3":
        required_major = 3
    else:
        raise ValueError("Unrecognized python interpreter: {}".format(
            REQUIRED_PYTHON))

    if system_major != required_major:
        raise TypeError(
            "This project requires Python {}. Found: Python {}".format(
                required_major, sys.version))
    else:
        print(">>> Development environment passes all tests!")


if __name__ == '__main__':
    main()


>>> Development environment passes all tests!


In [21]:
print(REQUIRED_PYTHON)

python3


ImportError: attempted relative import with no known parent package

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# Sample corpus
corpus = ['This is the first document.',
          'This document is the second document.',
          'And this is the third one.',
          'Is this the first document?']

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the corpus
X = vectorizer.fit_transform(corpus)

# Get the feature names
feature_names = vectorizer.get_feature_names()

# Print the count vectorized representation
print(X.toarray())

# Print the feature names
print(feature_names)

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

In [1]:
# Vectorizing the data with Count Vectorizer 
bag_of_words = CountVectorizer(min_df=5)
inbound_cv = bag_of_words.fit_transform(string_processed_inbound)


NameError: name 'CountVectorizer' is not defined

### 2.TFIDF

In [ ]:
tfidf = TfidfVectorizer(min_df=5, ngram_range = (1,3))
# Storing tfidf data and transforming them into sparse matrices
inbound_tfidf = tfidf.fit_transform(string_processed_inbound)
inbound_tfidf

## Pretrained word embeddings
While bag-of-words approaches can capture the presence of words in a text, they fail to preserve the crucial aspect of word order. This limitation necessitates the exploration of more sophisticated text vectorization techniques. With the advent of advanced methods, we can now effectively encode intent clusters into meaningful vector representations.

#### 3. Glove 

I'll explore various word embeddings to encode text differently and assess their effectiveness, beginning with GloVe word embeddings—an unsupervised learning algorithm for generating word vector representations.

Gensim simplifies the use of pretrained word embeddings, offering a specialized data format for easy loading into a numpy array.

Note: I won't utilize this method because my clustering algorithms require tweets to represent single points, while this method transforms individual words. I'm keeping this note in my notebook to track progress.


In [52]:
%%time
with open("C:\\Sagar Study\\ML and Learning\\Projects\\customer-support-bot\\amazon_customer_support\\objects\\glove.6B.50d.txt", "r", encoding="utf-8") as file: 
    word_to_vec_map = {} 
    for line in file: 
        values = line.split() 
        curr_word = values[0] 
        coefs = np.array(values[1:], dtype=np.float32)  # Fix: Convert values to float
        word_to_vec_map[curr_word] = coefs 

CPU times: total: 2.12 s
Wall time: 4.5 s


In [53]:
word_to_vec_map["enemy"]

array([ 0.70153 , -0.43853 ,  1.0509  , -0.33431 ,  0.67151 , -0.17677 ,
        0.80079 ,  0.90158 , -0.29513 , -0.65586 , -0.083404,  0.35146 ,
       -0.41071 ,  0.29446 , -1.1955  , -0.45611 ,  0.56877 ,  0.073522,
       -1.2616  ,  0.22276 , -0.57735 ,  0.12075 ,  0.54712 , -0.34094 ,
        0.2164  , -1.804   , -0.70362 , -0.56337 ,  1.8773  ,  0.1301  ,
        2.271   , -0.25882 , -0.46309 , -0.7759  , -0.22926 ,  0.62156 ,
       -0.043353, -0.60943 , -1.6791  , -0.018271,  0.53893 , -0.50689 ,
        0.88454 , -0.11158 ,  0.57013 , -0.69098 , -0.43072 , -0.45332 ,
       -0.27984 , -0.056133], dtype=float32)

In [28]:
word_to_vec_map["Sea"]

KeyError: 'Sea'

As for interpretation later on, since this being the main embedding algorithm in my pipeline, 

words_to_vec_map["sea"]

## 4. Doc2Vec
Since this is the main embedding method I will use for my pipeline, I display how I capitalized this embedding in the next notebook. {to be edited later on}

## 5. Hugging Face
This is a startup that does a lot with NLP. I explore their encoders.

BERT wouldn't really be a good option because a large part of that was trained with Wikipedia data.

I am not sure what doc2vec is trained on, I think my results will be better if I find a Twitter based word embedding!

## 6. Fast-text 


An embedding methodology developed by Facebook AI team

This one as well, like BERT, doesn't seem to be a good option at first glance due to its dependence on Wikipedia for training. 

Nevertheless, we will try and evaluate if it works well 

# Scaling the data 

Before we cluster, let's make scaled versions of our dataset first, which would be good for distance-based clustering methods. In general, these vectors shouldn't really need scaling, but it may help for computational purposes. I only do this for my count vectorized and tfidf vectorized data, not ones with the more meaningful word embeddings.

In [ ]:
# Fitting and transforming to create standard scald versions of the data through the use of MaxAbsScaler
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
inbound_cv = scaler.fit_transform(inbound_cv)
outbound_cv = scaler.fit_transform(outbound_cv)

In [ ]:
inbound_cv.head()

In [ ]:
outbound_cv.head()

# Tweet Collection With Clustering

Training Consideration: Because I will be working with neural networks in the next phase of my project, my objective is to identify approaximately `x` distinct intents in the data through Exploratory Data Analysis(EDA). Each of these should ideally consist of 1000 tweets similar to it to generate your training data for that intent with Gensim's `model.docvecs...` (complete next) function

I aim to employ clustering methods and topic modeling techniques on my dataset to extract prominent themes, intending to manually assign labels to these clusters.

Achieving successful outcomes will be more feasible if my dataset pertains uniformly to the domain of customer service. I've taken care to ensure that this Twitter data related to Amazon primarily focuses on this domain. This careful curation aims to enable my upcoming model to capture the subtleties required for intent classification. In essence, the conversational efficacy of the bot largely depends on the language it's been trained on.

With respect to the nature of clustering algorithms, I understand that they may not precisely group the data as anticipated beforehand. It's an algorithmic process and might not seamlessly cluster intents together as desired. However, I will back my curiosity to discover insightful trends - if found any!!!

## <font color='blue'>1. K Means</color>
My first approach for the clustering my word vectors is K-Means, which tends to perform well on blobs.

A drawback is that it is very slow, and picking the value for K is hard - I don't even know how many intents there are in the data. This is why I start with larger jumps of K to get a higher level idea of which performs the best, then I dive deeper to finally decide what K works the best for finding the optimal number of intents in my dataset.

## 1.1. K-Means for my TFIDF and Count Vectorized Data
First I cluster the TFIDF and Count Vectorized data. Honestly, I won't really be expecting good results from it, so I won't spend a lot of effort doing this actual clustering on these two. But it's worth a shot to demonstrate an older and suboptimal approach.


I'm currently running the K-Means algorithm on my entire dataset. Alongside this, I'm performing hyperparameter optimization specifically on the n_clusters parameter. The initial progress bar reflects its progress with the dataset, while the second bar shows its completion concerning different values of n_clusters.

The process of applying K-Means across 10 iterations spanning from 10 to 100 for both data types consumed approximately three hours before the scaling step. Thankfully, post-scaling, the training procedure noticeably sped up.

Below this section, I've saved my results using Python's serialization package called Pickle to avoid the need to rerun the process again!

In [ ]:
%%time
# My grand dictionaries that will store all my results
wcss_grand = {}
labels_grand = {}
silhouette_scores_grand = {}
n_clusters = [10,20,30,40,50,60,70,80,90,100]

# Iterating through all the differently embedded data
for i,j in tqdm(enumerate(vectorized_data.items())): 
    name = j[0] # Here j[0] is the name of the dataset
    dataset = j[1] # And j[1] is the actual data
    
    # I store my metrics at these following lists
    wcss = []
    labels = []
    silhouette_scores = []
    
    # Looping through values of k
    for k in tqdm(n_clusters):    
        print(f'Currently fitting {name} with {k} clusters... Please wait')
        
        # Initializing with k-means++ ensures that you get don’t fall into the random initialization trap.
        kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state = 10)
        kmeans.fit(dataset)
        wcss.append(kmeans.inertia_)
        
        # Getting the silhouette score
        labels.append(kmeans.labels_)
        silhouette_scores.append(silhouette_score(dataset, kmeans.labels_))
        
        # Saving the models
        filename = f'models/kmeans/{name}-{k}neighbors.sav'
        joblib.dump(kmeans, filename)
        
    # Updating grand dictionary
    wcss_grand[name + '_wcss'] = wcss
    labels_grand[name + '_labels'] = labels
    silhouette_scores_grand[name + '_silhouettes'] = silhouette_scores

# Saving all my results
with open('objects/wcss_grand.pkl', 'wb') as handle:
    pickle.dump(wcss_grand, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('objects/labels_grand.pkl', 'wb') as handle:
    pickle.dump(labels_grand, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('objects/silhouette_scores_grand.pkl', 'wb') as handle:
    pickle.dump(silhouette_scores_grand, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Reading back in the results

In [ ]:
# Storing it into objects I can use in this notebook

with open('objects/wcss_grand.pkl', 'rb') as handle: # Change path 
    wcss_grand = pkl.load(handle)
with open('objects/labels_grand.pkl','rb') as handle: # Cnange path
    labels_grand = pkl.load(handle)
with open('objects/silhouette_scores_grand.pkl','rb') as handle: # Change path 
    silhouette_scores_grand = pkl.load(handle)

## Finding the Best K-Means Model
I will try creating an elbow plot to see if there exists a clear elbow. I will do so on both the **count vectorized** and **tfidf** features

In [ ]:
# Elbow Plot count vectorized
plt.figure(figsize=(8,5))
plt.plot(range(10, 101, 10), wcss_grand['inbound_cv_ma_wcss'], color = 'magenta')
plt.title('Elbow Method (Count Vectorized)')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Elbow Plot tfidf
plt.figure(figsize=(8,5))
plt.plot(range(10, 101, 10), wcss_grand['inbound_tfidf_ma_wcss'], color = 'magenta')
plt.title('Elbow Method (TFIDF)')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Silouette Plot count vectorized
plt.figure(figsize=(8,5))
plt.plot(range(10, 101, 10), silhouette_scores_grand['inbound_cv_ma_silhouettes'], color = 'red')
plt.title('Silhouette Method (Count Vectorized)')
plt.xlabel('Silhouette Score')
plt.ylabel('WCSS')
plt.show()

# Silouette Plot tfidf
plt.figure(figsize=(8,5))
plt.plot(range(10, 101, 10), silhouette_scores_grand['inbound_tfidf_ma_silhouettes'], color = 'red')
plt.title('Silhouette Method (TFIDF)')
plt.xlabel('Silhouette Score')
plt.ylabel('WCSS')
plt.show()

We see that these plots don't really chaneg much between the TFIDF and Count Vectorized data, furthering my statement above that they aren't really going to be the most useful 

## Visualizing my clusters with t-SNE 

I try different color maps and choose one so its easier to distinguish between clusters 

Available sequential colormaps:
```['viridis', 'plasma', 'inferno', 'magma', 'cividis']```

```['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']```

Available qualitative colormaps:
```['Pastel1', 'Pastel2', 'Paired', 'Accent',
                        'Dark2', 'Set1', 'Set2', 'Set3',
                        'tab10', 'tab20', 'tab20b', 'tab20c']```

In [ ]:
# Check out the shape of current data 
inbound_cv.shape, inbound_tfidf_ma.shape 


T-SNE is a probabilistic model, so it will take some time to run, especially because we have a large no of rows 

In [ ]:
%%time 
# Instantiate t-SNE
tsne = TSNE(n_components=2, random_state=1, n_jobs=-1)

# Fit t-SNE
inbound_cv_ma_tsne = tsne.fit_transform(inbound_cv_ma)

In [ ]:
# Plotting my visualization for each of my n_neighbors with my count vectorized data

for k in range(10,101,10):
    # Getting the right K-Means cluster labels.
    labels = joblib.load(f'models/kmeans/inbound_cv_ma-{str(k)}neighbors.sav').labels_ # See where these labels are saved 
    
    # Visualize high-dimensional data
    plt.figure(figsize=(13,12))
    plt.scatter(inbound_cv_ma_tsne[:,0], inbound_cv_ma_tsne[:,1], s=20, c = labels, cmap = 'magma')
    plt.title(f'2-D t-SNE Representation of my Count Vectorized Inbound data with {k} Clusters')
    plt.xlabel('t-SNE dimension 1')
    plt.ylabel('t-SNE dimension 2')
    plt.show()

- For now, we are seeing so many clusters embedded into each other without any clear demarcation between them. Should I play with `perspective` parameter to gain any more insights? 
- Also, at this stage it would be difficult to observe clusters separately since there are many dimensions and we are representing them on a 2D space 

In [ ]:
# Plotting my visualization for each of my n_neighbors with my tfidf data

for k in range(10,101,10):
    # Getting the right K-Means cluster labels.
    labels = joblib.load(f'models/kmeans/inbound_tfidf_ma-{str(k)}neighbors.sav').labels_
    
    # Visualize high-dimensional data
    plt.figure(figsize=(13,12))
    plt.scatter(inbound_tfidf_ma_tsne[:,0], inbound_tfidf_ma_tsne[:,1], s=20, c = labels, cmap = 'magma')
    plt.title(f'2-D t-SNE Representation of my TFIDF Inbound data with {k} Clusters')
    plt.xlabel('t-SNE dimension 1')
    plt.ylabel('t-SNE dimension 2')
    plt.show()

There doesn't seem to be much difference in the outputs of CountVectorized clusters and TFIDF clusters. Let's check for Doc2Vec and other semantic embeddings

## 1.2. K-Means for my Doc2Vec data

Notice that I did not scale my d2v data on purpose because I do not want to skew the distances that the pretrained model created.

In [ ]:
# Vectorized data
vectorized_data = {'inbound_cv_d2v': inbound_d2v}

# Briefly showing the contents of i and j
for i, j in enumerate(vectorized_data.items()):
    print(i, j)

In [ ]:
# My d2v dictionaries that will store all my results
wcss_d2v = {}
labels_d2v = {}
silhouette_scores_d2v = {}
n_clusters = [10,20,30,40,50,60,70,80,90,100]

# Iterating through all the differently embedded data
for i,j in tqdm(enumerate(vectorized_data.items())): 
    name = j[0] # Here j[0] is the name of the dataset
    dataset = j[1] # And j[1] is the actual data
    
    # I store my metrics at these following lists
    wcss = []
    labels = []
    silhouette_scores = []
    
    # Looping through values of k
    for k in tqdm(n_clusters):    
        print(f'Currently fitting {name} with {k} clusters... Please wait')
        
        # Initializing with k-means++ ensures that you get don’t fall into the random initialization trap.
        kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state = 10)
        kmeans.fit(dataset)
        wcss.append(kmeans.inertia_)
        
        # Getting the silhouette score
        labels.append(kmeans.labels_)
        silhouette_scores.append(silhouette_score(dataset, kmeans.labels_))
        
        # Saving the models
        filename = f'models/kmeans/{name}-{k}neighbors.sav'
        joblib.dump(kmeans, filename)
        
    # Updating d2v dictionary
    wcss_d2v[name + '_wcss'] = wcss
    labels_d2v[name + '_labels'] = labels
    silhouette_scores_d2v[name + '_silhouettes'] = silhouette_scores

# Saving all my results, now with a d2v tag
with open('objects/wcss_d2v.pkl', 'wb') as handle:
    pickle.dump(wcss_d2v, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('objects/labels_d2v.pkl', 'wb') as handle:
    pickle.dump(labels_d2v, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('objects/silhouette_scores_d2v.pkl', 'wb') as handle:
    pickle.dump(silhouette_scores_d2v, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Reading back in the results.

In [ ]:
# Storing it into objects I can use in this notebook

with open('objects/wcss_d2v.pkl', 'rb') as handle:
    wcss_d2v = pickle.load(handle)
with open('objects/labels_d2v.pkl','rb') as handle:
    labels_d2v = pickle.load(handle)
with open('objects/silhouette_scores_d2v.pkl','rb') as handle:
    silhouette_scores_d2v = pickle.load(handle)

Here are my plots: 

In [ ]:
# Elbow Plot d2v
plt.figure(figsize=(8,5))
plt.plot(range(10, 101, 10), wcss_d2v['inbound_cv_d2v_wcss'], color = 'magenta')
plt.title('Elbow Method (Doc2Vec)')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Silouette Plot d2v
plt.figure(figsize=(8,5))
plt.plot(range(10, 101, 10), silhouette_scores_d2v['inbound_cv_d2v_silhouettes'], color = 'red')
plt.title('Silhouette Method (Doc2Vec)')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.show()

Great, now we see that we have a slightly higher silhouette score which isn't negative now. It looks like K = 20 would be the best in this case as it has the highest silhouette score and there is sort of an elbow in the elbow plot, definitely more than at 80 clusters where it seems to be completely smooth.

In [ ]:
%time
# Instantiate t-SNE
tsne = TSNE(n_components=2, random_state=1, n_jobs=-1)

# Fit t-SNE
inbound_d2v_tsne = tsne.fit_transform(inbound_d2v)

In [ ]:
# Plotting my visualization for each of my n_neighbors, now with Doc2Vec embedded data

for k in range(10,101,10):
    # Getting the right K-Means cluster labels.
    labels = joblib.load(f'models/kmeans/inbound_cv_d2v-{str(k)}neighbors.sav').labels_
    
    # Visualize high-dimensional data
    plt.figure(figsize=(13,12))
    plt.scatter(inbound_d2v_tsne[:,0], inbound_d2v_tsne[:,1], s=20, c = labels, cmap = 'magma')
    plt.title(f'2-D t-SNE Representation of my Doc2Vec Inbound data with {k} Clusters')
    plt.xlabel('t-SNE dimension 1')
    plt.ylabel('t-SNE dimension 2')
    plt.show()

It's super hard to tell based on a t-SNE plot, so for that reason, I will evaluate how it clustered at a later section by actually looking at the labels!

## <font color = 'blue'>2. LDA (Latent Dirichlet Allocation) </color>
My second approach for the clustering is LDA topic modelling. It basically takes your data and splits it into topics. My goal is still to cluster, but with this method I hope to get more useful, distinct topics.

Useful articles:
* https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
* https://medium.com/nanonets/topic-modeling-with-lsa-psla-lda-and-lda2vec-555ff65b0b05

There are also newer, deep-learning based methods called LDA2Vec which could be interesting to explore as well.

However, due to prorities shifting, I will employ this step as a future step.

In [ ]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
 
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 5
number_words = 10
# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

In [ ]:
%%time
from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis
LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(number_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
LDAvis_prepared = sklearn_lda.prepare(lda, count_data, count_vectorizer)
with open(LDAvis_data_filepath, 'w') as f:
        pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath) as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(number_topics) +'.html')

For time constraint's sake, I decided not to use DBScan because they will achieve a clustering result similar to K-Means. I could have also use Gaussian Mixed Models or Heirarchical Clustering to achieve this clustering result. **Use GMMs and analyze your result on them as well**

## Finding and Visualizing Intent Differences Between Clusters

I saved all my models in a folder in this directory called folder. All that I have to do is pick a hyperparamater setting for that model and visualize what words are in those clusters.

In [ ]:
processed_inbound

I think it's really useful to see the top 10 words in a cluster to get a good idea of what the intents in that cluster is! (Check out the intents...Edit sentences in the final notebook)